In [1]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv
from sympy import content

# API KEY 정보로드
load_dotenv()

True

In [3]:
# LCEL for chain
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_template("tell me a short joke about {topic}")

model = ChatOpenAI(
    model="gpt-4o-mini"
)

# StrOutputParser() ai 의 응답을 내용만 봅아서 볼 수 있음. 원래는 메타 데이터도 포함됨.
chain = prompt | model | StrOutputParser()

print(f"[답변]: {chain.invoke({'topic': 'ice cream'})}")



[답변]: What did the ice cream cone say to the refrigerator?  

"Stop looking at me, I'm just trying to chill!"


In [4]:
# chain 의 stream 기능 써보기
model = ChatOpenAI(
    model="gpt-4o-mini"
)

prompt = ChatPromptTemplate.from_template("tell me a short joke about {topic}")

chain = prompt | model

for s in chain.stream({"topic": "ice cream"}):
    print(s.content, end="", flush=True)

What did the ice cream say to the cake? 

"You're so sweet, but I’m just chilling!" 🍦🍰


Stream 사용 이유
* 빠른 응답 제공: 전체 응답이 생성될 때까지 기다릴 필요 없이, 한 토큰씩 받아볼 수 있음.
* 실시간 처리 가능: UI에서 점진적으로 텍스트를 표시하거나, 스트림 데이터를 다른 곳에서 활용할 수 있음.
* 메모리 효율성: 한 번에 모든 데이터를 로드하지 않기 때문에 메모리 부담이 줄어듦.

In [6]:
from langchain_core.messages import SystemMessage
from langchain_core.prompts import ChatPromptTemplate, HumanMessagePromptTemplate

chat_template = ChatPromptTemplate.from_messages(
    [
        SystemMessage(
            content=(
                "너는 소설 전문가 AI야. 사용자가 원하는 장르의 영화를 리스트 형태로 추천해줘."
                         'ex( Query: 현대 문학 소설 5개 추천해줘 / 답변: ["aa", "ee", "ee2"]'
            )
        ),
        HumanMessagePromptTemplate.from_template("{text}"),
    ]
)

model = ChatOpenAI(model="gpt-4o-mini")
chain = chat_template | model
chain.invoke("사회")

AIMessage(content='사회 관련 소설 5개 추천해드릴게요:\n\n1. "1984" - 조지 오웰\n2. "브레이브 뉴 월드" - 올더스 헉슬리\n3. "민족의 상처" - 하정우\n4. "지상의 노래" - 박완서\n5. "신의 탑" - SIU (웹툰)\n\n이 소설들은 사회 구조와 인간의 갈등, 그리고 사회의 본질을 탐구하는 내용을 담고 있습니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 116, 'prompt_tokens': 63, 'total_tokens': 179, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_06737a9306', 'finish_reason': 'stop', 'logprobs': None}, id='run-fc0b785a-3ef2-4971-aafa-8fdd0d6b5bf9-0', usage_metadata={'input_tokens': 63, 'output_tokens': 116, 'total_tokens': 179, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [8]:
# 부분적으로 프롬포트에 미리 넣어두고 싶은 부분이 있다면?
from langchain_core.prompts import PromptTemplate
from langchain.output_parsers import CommaSeparatedListOutputParser

# parser 선언
output_parser = CommaSeparatedListOutputParser()

# csv 파서 작동을 위한 형식 지정 프롬프트 로드
format_instructions = output_parser.get_format_instructions()

# 프롬포트 템플릿의 partial_variables 에 csv 형식 지정 프롬포트 주입
prompt = PromptTemplate(template="List {subject}. {format_instructions}", input_variables=["subject"],
                          partial_variables={"format_instructions": format_instructions}, )

model = ChatOpenAI(model="gpt-4o-mini")
chain = prompt | model | output_parser
chain.invoke({"subject":"한국 현대 문학"})



['김영하', '한강', '박완서', '윤고은', '백수린', '조남주', '김숨', '황정은', '정세랑', '손보미']